In [1]:
import os

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

import numpy as np

# = = = = = = = = = = = = = = =

import re
import html as html_lib

In [2]:
directory_in = './file/1.part_number'

list_file = [file for file in sorted(os.listdir(directory_in)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_in}/{file}')]
file = list_file[0]

file

'crawler_1.xlsx'

In [3]:
list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(f'{directory_in}/{file}',
                                                                       header=0,
                                                                       dtype=str).fillna('')

df_input['No'] = df_input['No'].astype(int)

count_remain = len(df_input)

print(f'总数量：{count_remain}')
print()
df_input

总数量：543



,No,Group,Type,Manufacturer,Part Number,Url,Type Code
0,1,Engine,Timing Chain & Component Kit,MELLING,30250S,"https://www.rockauto.com/en/parts/melling,3025...",5756
1,2,Engine,Timing Chain & Component Kit,MELLING,30250SX,"https://www.rockauto.com/en/parts/melling,3025...",5756
2,3,Engine,Timing Chain & Component Kit,MELLING,30420S,"https://www.rockauto.com/en/parts/melling,3042...",5756
3,4,Engine,Timing Chain & Component Kit,MELLING,30430S,"https://www.rockauto.com/en/parts/melling,3043...",5756
4,5,Engine,Timing Chain & Component Kit,MELLING,30460S,"https://www.rockauto.com/en/parts/melling,3046...",5756
...,...,...,...,...,...,...,...
538,539,Engine,Timing Chain & Component Kit,MELLING,TOPK1014,"https://www.rockauto.com/en/parts/melling,TOPK...",5756
539,540,Engine,Timing Chain & Component Kit,MELLING,TOPK1015,"https://www.rockauto.com/en/parts/melling,TOPK...",5756
540,541,Engine,Timing Chain & Component Kit,MELLING,TOPK1015HV,"https://www.rockauto.com/en/parts/melling,TOPK...",5756
541,542,Engine,Timing Chain & Component Kit,MELLING,TOPK1016,"https://www.rockauto.com/en/parts/melling,TOPK...",5756


In [4]:
crawler_status, (crawler_index, crawler_series) = False, next(df_input.iterrows())

crawler_status, crawler_index, crawler_series

(False,
 0,
 No                                                              1
 Group                                                      Engine
 Type                                 Timing Chain & Component Kit
 Manufacturer                                              MELLING
 Part Number                                                30250S
 Url             https://www.rockauto.com/en/parts/melling,3025...
 Type Code                                                    5756
 Name: 0, dtype: object)

In [5]:
import time

data = {'dopartsearch': '1',
        'partsearch[partnum][partsearch_007]': crawler_series['Part Number'],
        'partsearch[manufacturer][partsearch_007]': crawler_series['Manufacturer'],
        'partsearch[parttype][partsearch_007]': crawler_series['Type Code'],
        'partsearch[do][partsearch_007]': '1',
        'func': 'sendparttabsearch',
        'payload': '{}',
        'api_json_request': '1'}

count_retry = 0
while True:
    count_retry += 1
    print(count_retry)
    try:
        url_check = crawler_series['Url']

        resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                             data=data,
                             headers=get_header(),
                             proxies=get_proxy(),
                             timeout=(5, 15))

        if resp.status_code == 200:
            dict_ = resp.json()

            if 'redirect_to_url' not in dict_ and 'collected_javascript' in dict_:
                break
    except KeyboardInterrupt:
        break
    except:
        time.sleep(3)
        continue

dict_

1


{'searchnoderesults': '<div class="ranavnode" id="nav[__GIP__1__]"><input type="hidden" autocomplete="off" id="jsn[__GIP__1__]" value="{&quot;groupindex&quot;:&quot;__GIP__1__&quot;,&quot;tab&quot;:&quot;partsearch&quot;,&quot;idepth&quot;:0,&quot;parttype&quot;:&quot;5756&quot;,&quot;catalogname&quot;:&quot;MELLING&quot;,&quot;partnumber&quot;:&quot;30250S&quot;,&quot;nodetype&quot;:&quot;parttabsearch&quot;,&quot;expanded&quot;:true,&quot;loaded&quot;:true}" /><input type="hidden" autocomplete="off" name="navnodeunique[partsearch-5756]" id="navnodeunique[partsearch-5756]" value="__GIP__1__" /><div id="nav_o[__GIP__1__]" class="ranavouter"><div class="inner"><table class="tbl"  cellpadding="0" cellspacing="0"><tr><td class="niconspace nexpandedicon" id="navicon[__GIP__1__]"><a href="/en/parts/melling,30250S,timing+chain+&amp;+component+kit,5756" onclick="cataloglite.LinkIntercept_ToggleNavNode(&quot;__GIP__1__&quot;); return false;" tabindex="-1"><img src="/Images/1pxtransparent.png" 

In [6]:
soup = BeautifulSoup(dict_['searchnoderesults'], 'lxml')
html = etree.HTML(str(soup))

with open('./part.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

<html>
 <body>
  <div class="ranavnode" id="nav[__GIP__1__]">
   <input autocomplete="off" id="jsn[__GIP__1__]" type="hidden" value='{"groupindex":"__GIP__1__","tab":"partsearch","idepth":0,"parttype":"5756","catalogname":"MELLING","partnumber":"30250S","nodetype":"parttabsearch","expanded":true,"loaded":true}'/>
   <input autocomplete="off" id="navnodeunique[partsearch-5756]" name="navnodeunique[partsearch-5756]" type="hidden" value="__GIP__1__"/>
   <div class="ranavouter" id="nav_o[__GIP__1__]">
    <div class="inner">
     <table cellpadding="0" cellspacing="0" class="tbl">
      <tr>
       <td class="niconspace nexpandedicon" id="navicon[__GIP__1__]">
        <a href="/en/parts/melling,30250S,timing+chain+&amp;+component+kit,5756" onclick='cataloglite.LinkIntercept_ToggleNavNode("__GIP__1__"); return false;' tabindex="-1">
         <img alt="Toggle" src="/Images/1pxtransparent.png"/>
        </a>
       </td>
       <td class="nlabel">
        <a class="navlabellink nvoffset nnor

In [7]:
list_tbody = html.xpath('//tbody[contains(@id, "listingcontainer[__GIP__")]')

list_tbody

[<Element tbody at 0x120147fc0>]

In [8]:
list_dict = [json.loads(tbody.xpath('./tr[1]/td[contains(@class, "listing-inner-content")]/input[contains(@id, "listing_data_essential[__GIP__")]/@value')[0]) for tbody in list_tbody]

list_dict

[{'groupindex': '__GIP__3__',
  'carcode': 0,
  'parttype': '5756',
  'partkey': '8304960',
  'opts': {'0-0-0-1': {'warehouse': '72901',
    'whpartnum': 'MEL 3-0250S@NK',
    'optionlist': '0',
    'paramcode': '0',
    'notekey': '0',
    'multiple': '1'}}}]

In [9]:
list_type_code, list_part_code = [dict_['parttype'] for dict_ in list_dict], [dict_['partkey'] for dict_ in list_dict]

list_type_code, list_part_code

(['5756'], ['8304960'])

In [10]:
array_index_type_code = np.where(np.array(list_type_code) == crawler_series['Type Code'])[0]

array_index_type_code

array([0])

In [11]:
list_dict_2 = [json.loads(tbody.xpath('./tr[1]/td/input[contains(@id, "listing_data_supplemental[__GIP__")]/@value')[0]) for tbody in list_tbody]

list_dict_2

[{'partnumber': '30250S',
  'catalogname': 'MELLING',
  'belongstolisting': '__GIP__2__',
  'sortgroup': 0,
  'sortgrouptext': '',
  'paramdesc': '',
  'iskitparent': True}]

In [12]:
list_part_number, list_manufacturer, list_choose = [dict_['partnumber'] for dict_ in list_dict_2], [dict_['catalogname'] for dict_ in list_dict_2], [dict_['paramdesc'].strip() for dict_ in list_dict_2]

list_part_number, list_manufacturer, list_choose

(['30250S'], ['MELLING'], [''])

In [13]:
array_index_part_number, array_index_manufacturer = np.where(np.array(list_part_number) == crawler_series['Part Number'])[0], np.where(np.array(list_manufacturer) == crawler_series['Manufacturer'])[0]

array_index_part_number, array_index_manufacturer

(array([0]), array([0]))

In [14]:
array_index_part = np.intersect1d(np.intersect1d(array_index_type_code, array_index_part_number), array_index_manufacturer)

array_index_part

array([0])

In [15]:
if array_index_part.size == 1:
    index_part = array_index_part[0]

index_part

np.int64(0)

In [16]:
list_span = list_tbody[index_part].xpath('./tr[1]/td[contains(@class, "listing-inner-content")]/div[@class="listing-text-row-moreinfo-truck"]/span[@class="span-link-underline-remover"]')
list_note = [etree.tostring(span).decode('utf-8') for span in list_span]
list_note = [re.sub(r'<.*?>', '', note) for note in list_note]
list_note = [re.sub(r' +', ' ', note.strip()) for note in list_note]
list_note = [html_lib.unescape(note) for note in list_note]

list_note

[]

In [17]:
dict_note, i = {}, 0
for note in list_note:
    if note != '':
        dict_note[str(i)] = note

dict_note

{}

In [18]:
list_a = list_tbody[index_part].xpath('./tr[1]/td[contains(@class, "listing-inner-content")]/div[@class="listing-text-row-moreinfo-truck"]/a[@class="ra-btn ra-btn-moreinfo"]')
info = list_a[0].xpath('./@href')[0].strip() if list_a else ''
info = info if info.startswith('https://www.rockauto.com/en/moreinfo.php?') else f'https://www.rockauto.com/en/moreinfo.php?pk={list_part_code[index_part]}&pt={list_type_code[index_part]}&Lennon=1'

info

'https://www.rockauto.com/en/moreinfo.php?pk=8304960&cc=0&pt=5756'

In [19]:
list_div = list_tbody[index_part].xpath('./tr[1]/td[contains(@class, "listing-inner-content")]/div[@class="listing-text-row"]/span[contains(@title, "Please make sure")]/parent::div[@class="listing-text-row"]')
list_note = [etree.tostring(div).decode('utf-8') for div in list_div]
list_note = [re.sub(r'<.*?>', '', note) for note in list_note]
list_note = [re.sub(r' +', ' ', note.strip()) for note in list_note]
list_note = [html_lib.unescape(note) for note in list_note]

list_note

['Category: Timing Chain & Component Kit']

In [20]:
dict_note_2, i = {}, 0
for note in list_note:
    if note != '':
        dict_note_2[str(i)] = note

dict_note_2

{'0': 'Category: Timing Chain & Component Kit'}

In [21]:
list_list_text = [text.split(',') for td in list_tbody[index_part].xpath('./tr[1]/td[contains(@class, "listing-inner-content")]') for text in td.xpath('./span[contains(@title, "Replaces these Alternate")]/text()')]
list_text = [text.strip() for list_text in list_list_text for text in list_text if text.strip() != '']
oem = ';'.join(list(dict.fromkeys(list_text)))

oem

''

In [22]:
list_input = list_tbody[index_part].xpath('./tr[1]/td/div[contains(@class, "listing-image-space-occupy")]/input[contains(@id, "jsninlineimg[__GIP__")]')
list_dict = [json.loads(value) for input_ in list_input for value in input_.xpath('./@value')]

list_dict

[{'Size': 'Thumb',
  'SlotIndex': 0,
  'Slots': [{'TitlePhrase': None,
    'ParamCodes': [None],
    'Type': 'Single',
    'ImageData': {'Thumb': '/info/583/3-0250S-1-KIT__ra_m.jpg',
     'Popup': '/info/583/3-0250S-1-KIT.jpg',
     'Full': '/info/583/3-0250S-1-KIT.jpg'}},
   {'TitlePhrase': '<b>[Timing Chain Guide / Damper]</b>',
    'ParamCodes': [None],
    'Type': 'Single',
    'ImageData': {'Thumb': '/info/583/BT7015-2-SID__ra_m.jpg',
     'Popup': '/info/583/BT7015-2-SID.jpg',
     'Full': '/info/583/BT7015-2-SID.jpg'}},
   {'TitlePhrase': '<b>[Timing Chain Guide / Damper]</b>',
    'ParamCodes': [None],
    'Type': 'Single',
    'ImageData': {'Thumb': '/info/583/BG5515-1-SID__ra_m.jpg',
     'Popup': '/info/583/BG5515-1-SID.jpg',
     'Full': '/info/583/BG5515-1-SID.jpg'}},
   {'TitlePhrase': '<b>[Timing Chain]</b>',
    'ParamCodes': [None],
    'Type': 'Single',
    'ImageData': {'Thumb': '/info/583/715F-1-ANG__ra_m.jpg',
     'Popup': '/info/583/715F-1-ANG.jpg',
     'Full': 

In [23]:
dict_src, i = {}, 0
for dict_ in list_dict:
    list_src = dict_.get('Slots', [])
    for src in list_src:
        dict_src[str(i)] = f'''https://www.rockauto.com{src['ImageData']['Full'].strip()}'''
        i += 1

dict_src

{'0': 'https://www.rockauto.com/info/583/3-0250S-1-KIT.jpg',
 '1': 'https://www.rockauto.com/info/583/BT7015-2-SID.jpg',
 '2': 'https://www.rockauto.com/info/583/BG5515-1-SID.jpg',
 '3': 'https://www.rockauto.com/info/583/715F-1-ANG.jpg',
 '4': 'https://www.rockauto.com/info/583/S906A-1-FRO.jpg',
 '5': 'https://www.rockauto.com/info/583/BT5514-1-ANG.jpg',
 '6': 'https://www.rockauto.com/info/583/717F-1-ANG.jpg',
 '7': 'https://www.rockauto.com/info/583/BT7016-1-TOP.jpg',
 '8': 'https://www.rockauto.com/info/583/S899-1-FRO.jpg',
 '9': 'https://www.rockauto.com/info/583/S905-1-FRO.jpg',
 '10': 'https://www.rockauto.com/info/583/BG7019-2-SID.jpg'}

In [24]:
list_span = list_tbody[index_part].xpath('./tr[1]/td[contains(@id, "listingtd[__GIP__") and contains(@id, "__][price]")]/span[contains(@id, "dprice[__GIP__") and contains(@id, "__][td]")]/span/span[contains(@id, "dprice[__GIP__") and contains(@id, "__][v]")]')
list_price = [etree.tostring(span).decode('utf-8') for span in list_span]
list_price = [re.sub(r'<.*?>', '', price) for price in list_price]
list_price = [re.sub(r' +', ' ', price.strip()) for price in list_price]
list_price = [html_lib.unescape(price) for price in list_price]
dict_price = {'0': '; '.join(list_price)}

dict_price

{'0': '$171.79'}

In [25]:
list_td = list_tbody[index_part].xpath('./tr[1]/td[contains(@class, "listing-inner-content")]')
list_price = [etree.tostring(option).decode('utf-8') for td in list_td for option in td.xpath('./div/div[contains(@class, "listing-optionchoice-multiple-container")]/span[contains(@id, "ddrepl[optionchoice[__GIP__") and contains(@id, "__]][container]")]/span[@class="hide-if-js"]/select[contains(@id, "optionchoice[__GIP__") and contains(@id, "__]")]/option[@value!=""]')]
list_price = [re.sub(r'<.*?>', '', price) for price in list_price]
list_price = [re.sub(r' +', ' ', price.strip()) for price in list_price]
list_price = [html_lib.unescape(price) for price in list_price]
if list_price:
    dict_price['1'] = {}
    for i, price in enumerate(list_price):
        dict_price['1'][str(i)] = price

dict_price

{'0': '$171.79'}

In [26]:
list_dict_correct.append({'No': crawler_series['No'],
                          'Group': crawler_series['Group'],
                          'Type': crawler_series['Type'],
                          'Manufacturer': crawler_series['Manufacturer'],
                          'Part Number': crawler_series['Part Number'],
                          'OEM': oem,
                          'Picture': '',
                          'Url': crawler_series['Url'],
                          'Json_Src': json.dumps(dict_src),
                          'Info': info,
                          'Json_Note 1': json.dumps(dict_note),
                          'Json_Note 2': json.dumps(dict_note_2),
                          'Choose': list_choose[index_part],
                          'Json_Price': json.dumps(dict_price),
                          'Type Code': crawler_series['Type Code'],
                          'Part Code': list_part_code[index_part],
                          'JOIN_MPNTCPC': f'''{crawler_series['Manufacturer']};{crawler_series['Part Number']};{crawler_series['Type Code']};{list_part_code[index_part]}'''})

list_dict_correct

[{'No': 1,
  'Group': 'Engine',
  'Type': 'Timing Chain & Component Kit',
  'Manufacturer': 'MELLING',
  'Part Number': '30250S',
  'OEM': '',
  'Picture': '',
  'Url': 'https://www.rockauto.com/en/parts/melling,30250S,timing+chain+&+component+kit,5756',
  'Json_Src': '{"0": "https://www.rockauto.com/info/583/3-0250S-1-KIT.jpg", "1": "https://www.rockauto.com/info/583/BT7015-2-SID.jpg", "2": "https://www.rockauto.com/info/583/BG5515-1-SID.jpg", "3": "https://www.rockauto.com/info/583/715F-1-ANG.jpg", "4": "https://www.rockauto.com/info/583/S906A-1-FRO.jpg", "5": "https://www.rockauto.com/info/583/BT5514-1-ANG.jpg", "6": "https://www.rockauto.com/info/583/717F-1-ANG.jpg", "7": "https://www.rockauto.com/info/583/BT7016-1-TOP.jpg", "8": "https://www.rockauto.com/info/583/S899-1-FRO.jpg", "9": "https://www.rockauto.com/info/583/S905-1-FRO.jpg", "10": "https://www.rockauto.com/info/583/BG7019-2-SID.jpg"}',
  'Info': 'https://www.rockauto.com/en/moreinfo.php?pk=8304960&cc=0&pt=5756',
  'Json

In [27]:
crawler_status = True

crawler_status

True

In [28]:
crawler_series['Check_Url'] = url_check

list_series_error.append(crawler_series)

list_series_error

[No                                                              1
 Group                                                      Engine
 Type                                 Timing Chain & Component Kit
 Manufacturer                                              MELLING
 Part Number                                                30250S
 Url             https://www.rockauto.com/en/parts/melling,3025...
 Type Code                                                    5756
 Check_Url       https://www.rockauto.com/en/parts/melling,3025...
 Name: 0, dtype: object]

In [29]:
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['No'],
                                                             ascending=[True],
                                                             ignore_index=True)
    df_correct.to_excel('./test_part.xlsx', index=False)

df_correct

,No,Group,Type,Manufacturer,Part Number,OEM,Picture,Url,Json_Src,Info,Json_Note 1,Json_Note 2,Choose,Json_Price,Type Code,Part Code,JOIN_MPNTCPC
0,1,Engine,Timing Chain & Component Kit,MELLING,30250S,,,"https://www.rockauto.com/en/parts/melling,3025...","{""0"": ""https://www.rockauto.com/info/583/3-025...",https://www.rockauto.com/en/moreinfo.php?pk=83...,{},"{""0"": ""Category: Timing Chain & Component Kit""}",,"{""0"": ""$171.79""}",5756,8304960,MELLING;30250S;5756;8304960


In [30]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel('./test_part_error.xlsx', index=False)

df_error

,No,Group,Type,Manufacturer,Part Number,Url,Type Code,Check_Url
0,1,Engine,Timing Chain & Component Kit,MELLING,30250S,"https://www.rockauto.com/en/parts/melling,3025...",5756,"https://www.rockauto.com/en/parts/melling,3025..."
